# Data Facility 101

In [ ]:
# Import Postgres library
import psycopg2 as pg
import pandas as pd
import geopandas as gp

In [ ]:
# Create connection string
connStr = "dbname=df_spatial"

In [ ]:
# Open connection
conn = pg.connect(connStr)

In [ ]:
# Create cursor
cur = conn.cursor()

In [ ]:
# Execute a query
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type='BASE TABLE'")
print cur.fetchall()

## Use Pandas to query

In [ ]:
# Select number of Private Primary and Federal Primary jobs for NYC counties
qry = "SELECT county_fips, year, type, sum(c000) c000, sum(ca01) ca01, sum(ca02) ca02, sum(ca03) ca03 FROM (SELECT left(h_geocode, 5) county_fips, year, type, c000, ca01, ca02, ca03 FROM nyc_rac r WHERE type IN ('%s', '%s')) q GROUP BY year, type, county_fips ORDER BY type, county_fips, year;" % ('JT03', 'JT05')

In [ ]:
# Run query and create dataframe
df = pd.read_sql_query(qry, conn)

In [ ]:
# View info on data frame
df.info()

In [ ]:
# Get yearly percentage change for each category
df_percent_ch = df.groupby(['county_fips', 'type'])[['c000', 'ca01', 'ca02', 'ca03']].apply(lambda x: (x - x.shift(1)) / x.shift(1) * 100)

In [ ]:
# Concatenate with first 3 columns of original dataset
df_comb = pd.concat([df[['county_fips', 'year', 'type']], df_percent_ch], axis=1)

## Load Geometry using GeoPandas

In [ ]:
# Load geometric data into a GeoDataFrame for Manhattan
geo_df = gp.GeoDataFrame.from_postgis("SELECT * FROM tristate_blocks WHERE statefp10 || countyfp10 = '36061'", conn, geom_col='geom')

In [ ]:
geo_df.info()

In [ ]:
# Plot
%pylab inline
figure(figsize=(10,10))
geo_df.plot()